In [38]:
# 实时检测外部函数更新
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import os
# 1. 设置国内镜像源
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from bertopic import BERTopic
import torch # 新增：导入PyTorch（sentence_transformers的底层）
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

In [40]:
import pickle
import json
import numpy as np
import time
import pandas as pd


# 子数据集主题聚类参数设置

In [41]:
# ------------------------------------ 配置区 -------------------------------
sub_docs = [] # 存放子数据集
sub_timestamps = [] # 存放时间戳
target_indices = [] # 存放目标索引，方便获取词向量嵌入

# 版本控制
version = 'V2'
# 时间段控制
start_time = 2006
end_time = 2010
year_range = f'{start_time}-{end_time}'

# 数据源控制
data_source = 'GF专利'

model_name = "all-mpnet-base-v2"

# --- 配置区：UMAP 参数设置 ---
umap_params = {
    "n_neighbors": 15,
    "n_components": 5,
    "min_dist": 0.0,
    "metric": 'cosine',
    "random_state": 5,
    "low_memory": True
}

vectorizer_params = {
    "ngram_range": (1, 3), # 词组范围：1-3个词
    "min_df": 3, # 过滤低频噪声，对大样本非常重要
    "max_features": 100000 # 防止内存溢出
}

# --- HDBSCAN参数设置 ---
HDBSCAN_cfg = {
    "min_samples": 10,
    "metric": 'euclidean',
    "prediction_data": True
}

# HDBSCAN网格搜索范围
search_sizes = [15, 20, 30, 50, 70,100,150,200,250,300]
# 设定你期望的主题数量范围（例如 20 到 100 个）
min_expected_topics = 20
max_expected_topics = 50

# --- Step 2: 加载全量嵌入向量 ---
with open(r'results\embedding_results\embeddings_patents_zf_amb_slide_window2.pkl', 'rb') as f:
    full_embeddings = pickle.load(f)

system_prompt = """你是一名专业的国防专利与技术分析专家。
你的任务是根据提供的主題关键词，为每一个主题确定概括性名称。
要求：
1. 使用中文回答。
2. 名称格式为"编号：主题名称"，
示例如下
0: "1.武器装备结构设计与制图", 
1: "2.半导体工艺与微电子器件"

3. 输出JSON格式的字典，鍵为主题编号(数据类型为整型int)，值为主题名称(string)。
4. 主题简明，不要有多余的解释。"""

# 原始数据加载

In [42]:
# --- Step 1: 同步筛选文档和时间戳 ---
with open('data\patents_zf.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    
for i, item in enumerate(data):
    year = int(item['year'])
    if start_time <= year <= end_time:
        target_indices.append(i)
        sub_docs.append(item['combined_text'])
        # 这里的年份即为时间戳，BERTopic 支持整数年份或标准日期格式
        sub_timestamps.append(year) 

if len(sub_docs) == len(sub_timestamps):
    print(f'当前时间段为{start_time}-{end_time}年，共获取数据{len(sub_docs)}条，现在开始聚类……')
else:
    print('部分数据缺失年份信息，请检查原始数据')

print(sub_timestamps[:3])
for i in sub_docs[:3]: 
    print(i)

当前时间段为2006-2010年，共获取数据17163条，现在开始聚类……
[2006, 2007, 2006]
Human protein C or activated protein C prepd. using expression vector capable of integration in mammalian host cell DNA. Human protein C or activated protein C prepd. using expression vector capable of integration in mammalian host cell DNA. Human protein C or activated protein C prepd. using expression vector capable of integration in mammalian host cell DNA. DNA sequence codes for a protein having the same biological activity as human protein C (HPC) or human activated protein C(APC).Also claimed is an expression vector capable of integration in mammalian host cell DNA, including a promoter followed downstream by a nucleotide squence which encodes a protein having the same structure and/or activity as HPC or APC followed downstream by a polyadenylation signal, where transcription of the nucleotide sequence is directed by the promoter. Protein C plays a regulatory role in the coagulation process. The catalystic domain possesses 

In [43]:
# --- Step 3: 获取子集向量 ---
# 假设 full_embeddings 是 numpy 数组格式
sub_embeddings = full_embeddings[target_indices]

In [44]:
def load_stopwords(file_path):
    """从JSON文件加载停用词列表"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stopwords = json.load(f)
        print(f"已加载 {len(stopwords['common'])} 个停用词")
        return stopwords['common']  # 转换为集合提高查找效率
    except FileNotFoundError:
        print(f"错误：文件 {file_path} 不存在")
        return set()
    except json.JSONDecodeError:
        print(f"错误：文件 {file_path} 不是有效的JSON格式")
        return set()

In [45]:
# 1. 统一定义路径
local_model_path = f"./my_models/{model_name}"  # 固定路径

# 2. 检查并加载模型
if not os.path.exists(local_model_path):
    print(f"模型不存在，正在下载并保存到本地: {model_name}")
    # 直接从huggingface下载
    sentence_model = SentenceTransformer(model_name)
    # 保存到本地
    sentence_model.save(local_model_path)
    print(f"模型已保存至：{local_model_path}")
else:
    print(f"模型已存在，从本地加载: {local_model_path}")
    # 从本地加载
    sentence_model = SentenceTransformer(local_model_path)

模型已存在，从本地加载: ./my_models/all-mpnet-base-v2


# UMAP降维/HDBSCAN网格搜索

- 在专利聚类中，“噪声比例”不是唯一的指标，甚至不是最重要的指标。
- 我们的目标是在保持**主题可解释性（主题数适中）和噪声率（通常 20%-40% 是正常的）**之间找到平衡。

In [46]:
## UMAP降维
umap_model = UMAP(**umap_params) # 使用解包语法，方便调整
umap_embeddings = umap_model.fit_transform(sub_embeddings)
print('✔UMAP降维完成！现在开始进行HDBSCAN网格搜索寻找最佳min_cluster_size……')

best_topics = None
best_m_size = None
best_score = float('inf')

search_history = [] # 新增：用于记录网格搜索过程

print(f"{'Size':<10} | {'主题数':<8} | {'负样本数':<10} | {'噪声比例':<10} | {'耗时':<8}")
print("-" * 65)

for m_size in search_sizes:
    start_t = time.time()
    
    clusterer = HDBSCAN(**HDBSCAN_cfg,min_cluster_size=m_size)
    
    labels = clusterer.fit_predict(umap_embeddings)
    
    # 计算指标
    n_outliers = (labels == -1).sum() # 负样本数
    n_topics = len(set(labels)) - (1 if -1 in labels else 0)
    outlier_perc = n_outliers / len(labels)
    duration = time.time() - start_t

    # 保存历史记录
    res = {
        "min_cluster_size": m_size,
        "n_topics": n_topics,
        "n_outliers": n_outliers,
        "outlier_perc": f"{outlier_perc:.1%}",
        "duration": f"{duration:.1f}s",
        "is_best": False
    }
    search_history.append(res)

    # 筛选逻辑：首先要在合理的主题数范围内，然后选噪声最小的
    if min_expected_topics <= n_topics <= max_expected_topics:
        if outlier_perc < best_score:
            best_topics = n_topics
            best_score = outlier_perc
            best_m_size = m_size
    

    print(f"{m_size:<10} | {n_topics:<10} | {n_outliers:<12} | {outlier_perc:<12.1%} | {duration:<8.1f}s")
print("-" * 65)
if best_m_size is None:
    print("未在预设主题数范围内找到参数，建议调低 min_dist 或检查 UMAP 效果")
else:
    print(f"🏆 符合条件的负样本数最少的参数值: min_cluster_size = {best_m_size} (负样本比例: {best_score:.1%},主题数：{best_topics})")

# 标记最优参数
for item in search_history:
    if item["min_cluster_size"] == best_m_size:
        item["is_best"] = True

## HDBSCAN聚类
best_clusterer = HDBSCAN(**HDBSCAN_cfg,min_cluster_size=best_m_size)
labels = best_clusterer.fit_predict(umap_embeddings)

# 加载停用词
stop_words = load_stopwords('data\stopwords.json')
# 创建CountVectorizer模型,自定义停用词
vectorizer_model = CountVectorizer(
    **vectorizer_params,
    stop_words = stop_words
    )

# 初始化 BERTopic 时带上它
topic_model = BERTopic(
  embedding_model=sentence_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,       # 加上这一行
  hdbscan_model=best_clusterer # 加上这一行
)
#传入训练好的词向量，fit_transformer()包括了UMAP降维+HDBSCAN聚类，不建议这么做，可以将其拆开减少计算
#topics, probs = topic_model.fit_transform(docs, embeddings=embeddings,y = labels) 
#prob计算很花时间
topics, _ = topic_model.fit_transform(sub_docs, embeddings=sub_embeddings)
# 保存整个主题模型

save_path = rf"results\topic_models\bertopic_{data_source}_{year_range}_{version}"
topic_model.save(save_path)
print(rf"主题模型已保存到 results\topic_models\bertopic_{data_source}_{year_range}_{version}")

✔UMAP降维完成！现在开始进行HDBSCAN网格搜索寻找最佳min_cluster_size……
Size       | 主题数      | 负样本数       | 噪声比例       | 耗时      
-----------------------------------------------------------------
15         | 223        | 5810         | 33.9%        | 0.4     s
20         | 167        | 4923         | 28.7%        | 0.4     s
30         | 115        | 4461         | 26.0%        | 0.4     s
50         | 74         | 4372         | 25.5%        | 0.4     s
70         | 56         | 3917         | 22.8%        | 0.4     s
100        | 44         | 3779         | 22.0%        | 0.4     s
150        | 21         | 2110         | 12.3%        | 0.4     s
200        | 19         | 2489         | 14.5%        | 0.4     s
250        | 17         | 2916         | 17.0%        | 0.4     s
300        | 2          | 105          | 0.6%         | 0.4     s
-----------------------------------------------------------------
🏆 符合条件的负样本数最少的参数值: min_cluster_size = 150 (负样本比例: 12.3%,主题数：21)
已加载 175 个停用词


2026-01-21 17:46:21,292 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


主题模型已保存到 results\topic_models\bertopic_GF专利_2006-2010_V2


In [47]:
topic_model = BERTopic.load(rf"results\topic_models\bertopic_patents_zf_{year_range}_{version}")

In [48]:
# topic_model.get_topic_info()

In [49]:
# 获取所有主题的字典 {topic_id: [(word, score), ...]}
all_topics_dict = topic_model.get_topics()
# 过滤掉 -1 (噪声)，并将每个词列表转为逗号连接的字符串
clean_topics_dict = {
    topic_id: ",".join([word for word, score in words_list])
    for topic_id, words_list in all_topics_dict.items()
    if topic_id != -1
}

for item in clean_topics_dict.items():
    print(item)
    
# 检查结果
print(f'已获取{len(clean_topics_dict)}个主题的详细信息，现在通过AI判断主题名称……') 

(0, 'cell,acid,protein,sequence,comprising,comprises,cells,useful,amino,cancer')
(1, 'fluid,sample,sensor,surface,flow,analyte,device,channel,microfluidic,first')
(2, 'carbon,metal,material,coating,comprises,polymer,composite,surface,preferred,substrate')
(3, 'data,network,computer,system,method,involves,information,node,memory,nodes')
(4, 'image,light,optical,system,laser,beam,object,method,fiber,imaging')
(5, 'vehicle,projectile,system,explosive,assembly,underwater,end,robot,surface,device')
(6, 'hydrogen,gas,catalyst,water,stream,metal,fuel,comprises,reactor,oxide')
(7, 'optical,laser,waveguide,fiber,signal,light,photonic,output,layer,wavelength')
(8, 'layer,substrate,semiconductor,silicon,nitride,material,surface,gallium,device,silicon carbide')
(9, 'signal,circuit,output,digital,input,frequency,data,communication,system,voltage')
(10, 'ray,detector,radiation,imaging,image,tomography,system,gamma,rays,source')
(11, 'engine,heat,combustion,fuel,valve,gas,fluid,flow,cooling,air')
(12

In [50]:
# AI识别
user_prompt = f"请分析以下主題关键词，并返回 JSON 字典：\n{clean_topics_dict}"

In [51]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import re
# 加载 .env 文件
load_dotenv(".env")
API_KEY=os.environ.get('DEEPSEEK_API_KEY')
deepseek_chat_model = "deepseek-chat" # DeepSeek-V3.2的非思考模式
client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.deepseek.com")

In [52]:
# AI识别智能体参数设置：创建专门用于地址推理的 LLM 实例
Deepseek_reasponse = client.chat.completions.create(
    model=deepseek_chat_model,
    messages=[ # 对话消息列表
        {"role": "system", "content": system_prompt}, # 系统提示词，定义助手的行为
        {"role": "user", "content": user_prompt},
    ],
    response_format={'type': 'json_object'}, #强制json格式返回
    stream=False # 非流式响应（一次性返回完整结果）
)

In [53]:
content = json.loads(Deepseek_reasponse.choices[0].message.content)
#content = re.sub(r"^```json\s*|\s*```$", "", content.strip())
# 将键转为整型
formatted_labels = {int(k): v for k, v in content.items()}
print(formatted_labels)

# 使用自定义标签（需要先设置）
topic_model.set_topic_labels(formatted_labels)

# 降至二维，再可视化
reduced_embeddings = UMAP(n_neighbors=30, n_components=2, min_dist=0.0, metric='cosine').fit_transform(sub_embeddings)

{0: '生物技术与细胞工程', 1: '微流控与传感器技术', 2: '材料与涂层技术', 3: '计算机网络与信息系统', 4: '光学成像与激光系统', 5: '无人系统与弹药技术', 6: '氢能与催化反应技术', 7: '光子学与光波导技术', 8: '半导体材料与器件', 9: '数字电路与通信系统', 10: '辐射探测与成像技术', 11: '内燃机与热管理系统', 12: '燃料电池与电化学技术', 13: '涡轮发动机技术', 14: '电力与电机系统', 15: '有机发光器件技术', 16: '微机电系统（MEMS）', 17: '传感器与换能器技术', 18: '磁共振成像技术', 19: '天线与射频技术', 20: '医疗超声与生物电子技术', 21: '雷达与定位系统', 22: '磁存储与自旋电子技术'}


# 数据可视化

In [54]:
# 层次聚类
hierarchical_topics = topic_model.hierarchical_topics(sub_docs)

100%|██████████| 22/22 [00:00<00:00, 279.70it/s]


In [55]:
import os
from report_generator import generate_bertopic_report
# --- 0. 基础设置 ---
title_base = f'{year_range}年{data_source}数据Bertopic主题聚类结果一览表-{version}'
output_dir = f"BERTopic_Results_Subset_{year_range}_{data_source}_{version}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- 参数说明 ---
report_name = os.path.join(output_dir, f"report_{year_range}_{data_source}_{version}.html")
#generate_report(umap_params, search_history, best_m_size, model_name, report_name)

generate_bertopic_report(umap_cfg = umap_params, 
                         HDBSCAN_cfg=HDBSCAN_cfg,
                         vectorizer_cfg = vectorizer_params,
                         history= search_history,
                         best_size = best_m_size,
                         model_name = model_name, 
                         output_path = report_name)

# --- 1. 生成图表 (按照你想要的顺序) ---
#-------------------------------------------------------------------------
# [图 A] 主题关键词条形图 - 存为 barchart.html
fig_barchart = topic_model.visualize_barchart(
    top_n_topics=len(formatted_labels),
    custom_labels=True,
    n_words=10,
    height=400,
    title=f"{year_range}年{data_source}各主题关键词的c-TF-IDF权重得分条形图",
)

fig_barchart.update_layout(
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=36, color="black", weight="bold")
)
path_barchart = os.path.join(output_dir, "barchart.html")
fig_barchart.write_html(path_barchart)
#-------------------------------------------------------------------------
# [图 B] 层次聚类图 - 存为 hierarchy.html
fig_hierarchy = topic_model.visualize_hierarchy(
    hierarchical_topics=hierarchical_topics,
    custom_labels=True,
    title=f"{year_range}年{data_source}主题层次聚类图",
    height=800
)
fig_hierarchy.update_layout(
    title_x=0.5,
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_hierarchy = os.path.join(output_dir, "hierarchy.html")
fig_hierarchy.write_html(path_hierarchy)
#-------------------------------------------------------------------------
# [图 C] 文档分布散点图 - 存为 documents.html
fig_documents = topic_model.visualize_documents(
    #docs=[doc[:150] + "..." for doc in sub_docs],
    docs=[str(doc)[:150] + "..." for doc in sub_docs if doc is not None],
    reduced_embeddings=reduced_embeddings,
    custom_labels=True,
    hide_document_hover=False
)
# 更新散点图样式
fig_documents.update_layout(
    title=f"{year_range}年{data_source}主题分布图",
    title_x=0.5,
    width=1500,
    height=1200,
    margin=dict(l=80, r=80, t=100, b=80), # 设置对称边距
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_documents = os.path.join(output_dir, "documents.html")
fig_documents.write_html(path_documents)
#-------------------------------------------------------------------------
# [图 D] 主题时序图 - 存为 hierarchy.html
topics_over_time = topic_model.topics_over_time(sub_docs, sub_timestamps, global_tuning=False, evolution_tuning=False)
fig_topic_time = topic_model.visualize_topics_over_time(
    topics_over_time,
    custom_labels=True)
# 更新散点图样式
fig_topic_time.update_layout(
    title=f"{year_range}年{data_source}主题时序图",
    title_x=0.5,
    width=1500,
    height=800,
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_topic_time = os.path.join(output_dir, "topic_overtime_merged.html")
fig_topic_time.write_html(path_topic_time)
# --- 2. 创建导航索引页 (index.html) ---
# 注意：我在这里同步调整了按钮顺序和 iframe 的默认 src
index_content = f"""
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>{title_base}</title>
    <style>
        body {{ font-family: 'Microsoft YaHei', sans-serif; margin: 0; display: flex; flex-direction: column; height: 100vh; background-color: #f4f7f6; }}
        header {{ background: #2c3e50; color: white; padding: 15px 25px; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 5px rgba(0,0,0,0.2); }}
        h1 {{ margin: 0; font-size: 20px; }}
        nav {{ background: #ecf0f1; padding: 10px; display: flex; gap: 10px; border-bottom: 1px solid #ddd; }}
        .nav-btn {{ 
            padding: 8px 15px; background: white; border: 1px solid #bdc3c7; border-radius: 4px; 
            cursor: pointer; text-decoration: none; color: #34495e; font-size: 14px; transition: all 0.3s;
        }}
        .nav-btn:hover {{ background: #3498db; color: white; border-color: #2980b9; }}
        .nav-btn.active {{ background: #3498db; color: white; }}
        #content-frame {{ flex-grow: 1; border: none; width: 100%; }}
    </style>
</head>
<body>
    <header>
        <h1>{title_base}</h1>
        <span style="font-size: 12px; opacity: 0.8;">版本: {version}</span>
    </header>
    
    <nav>
        <a class="nav-btn" href="report_{year_range}_{data_source}_{version}.html" target="chart_frame">✨ {year_range}年{data_source}主题聚类参数说明</a>
        <a class="nav-btn" href="barchart.html" target="chart_frame">📈  {year_range}年{data_source}主题关键词权重图</a>
        <a class="nav-btn" href="hierarchy.html" target="chart_frame">📊  {year_range}年{data_source}主题层次聚类树状图</a>
        <a class="nav-btn" href="documents.html" target="chart_frame">📍  {year_range}年{data_source}主题分布散点图</a>
        <a class="nav-btn" href="topic_overtime_merged.html" target="chart_frame">⌚ {year_range}年{data_source}主题时序图</a>
    
    </nav>

    <iframe name="chart_frame" id="content-frame" src="report_{year_range}_{data_source}_{version}.html"></iframe>

    <script>
        const buttons = document.querySelectorAll('.nav-btn');
        buttons.forEach(btn => {{
            btn.addEventListener('click', function() {{
                buttons.forEach(b => b.classList.remove('active'));
                this.classList.add('active');
            }});
        }});
        // 默认高亮第一个按钮（即参数说明页面）
        buttons[0].classList.add('active');
    </script>
</body>
</html>
"""

with open(os.path.join(output_dir, f"index_{year_range}_{data_source}_res.html"), "w", encoding="utf-8") as f:
    f.write(index_content)

print(f"🎉 结果可视化已完成！已请打开文件夹: {output_dir}查看")

✅ 报告已成功生成至: BERTopic_Results_Subset_2006-2010_GF专利_V2\report_2006-2010_GF专利_V2.html
🎉 结果可视化已完成！已请打开文件夹: BERTopic_Results_Subset_2006-2010_GF专利_V2查看
